In [1]:
import pandas as pd
import numpy as np
import zipfile

df = pd.read_csv('text_data_mimic_iii_sample.csv')
# To read entire dataset as zip file.
# zf = zipfile.ZipFile('text_data_mimic_iii.csv.zip')
# df = pd.read_csv(zf.open('text_data_mimic_iii.csv'))

In [2]:
# To get the sample of the entire dataset and store that as csv
# x = no of rows you want
# df = df.sample(x).to_csv(file_name, encoding='utf-8')

We begin to work on the first 10000 data sets in the beginning just to get the structure working

In [3]:
data = df.as_matrix()
print(data)

[[1161046 1161046 1285489 ..., 'SUBDURAL HEMATOMA' 1 1]
 [1355922 1355922 1276636 ..., 'CORONARY ARTERY DISEASE' 1 1]
 [1263222 1263222 1737450 ..., 'NEWBORN' 0 1]
 ..., 
 [1354005 1354005 540197 ..., 'HEMOPTYSIS' 0 1]
 [1837377 1837377 1897346 ..., 'NEWBORN' 0 1]
 [608528 608528 1599315 ...,
  'ANXIETY\\BYPASS FEMORAL PEDAL ARTERY GRAFT; LEFT' 0 1]]


In [4]:
list(df)
for num, col_name in enumerate(df):
    print(num, col_name)

0 Unnamed: 0
1 Unnamed: 0.1
2 ROW_ID_x
3 SUBJECT_ID
4 HADM_ID
5 CHARTDATE
6 CHARTTIME
7 STORETIME
8 CATEGORY
9 DESCRIPTION
10 CGID
11 ISERROR
12 TEXT
13 ROW_ID_y
14 ADMITTIME
15 DISCHTIME
16 DEATHTIME
17 ADMISSION_TYPE
18 ADMISSION_LOCATION
19 DISCHARGE_LOCATION
20 INSURANCE
21 LANGUAGE
22 RELIGION
23 MARITAL_STATUS
24 ETHNICITY
25 EDREGTIME
26 EDOUTTIME
27 DIAGNOSIS
28 HOSPITAL_EXPIRE_FLAG
29 HAS_CHARTEVENTS_DATA


### SUBJECT_ID, HADM_ID
Identifiers which specify the patient: SUBJECT_ID is unique to a patient and HADM_ID is unique to a patient hospital stay.

### CG_ID 
This refers to the ID of the caregivere who made the observation and this is pretty irrelevant to us.

### STORE_TIME
This refers to the time at which the observation was actually recorded in the EHR. We shall use this time as CHART_TIME is not actually avalaible in the EHR.

### CATEGORY, DESCRIPTION
CATEGORY and DESCRIPTION define the type of note recorded. For example, a CATEGORY of ‘Discharge’ indicates that the note is a discharge note, and a DESCRIPTION of ‘Summary’ in conjunction with this indicates that the note is a discharge summary

### EDREGTIME, EDOUTTIME

Time that the patient was registered and discharged from the emergency department

### HOSPITAL_EXPIRE_FLG
Whether or not the patient deid in the hospital

### The columns that we consider are important for the initial part of the project
* 2 SUBJECT_ID
* 3 HADM_ID
* 6 STORETIME
* 7 CATEGORY
* 8 DESCRIPTION
* 11 TEXT
* 13 ADMITTIME
* 14 DISCHTIME
* 15 DEATHTIME
* 16 ADMISSION_TYPE
* 26 DIAGNOSIS
* 27 HOSPITAL_EXPIRE_FLAG

In [5]:
pat_dict = {}
# for rows in data:
#     pat_id = rows[2]
#     pat_dict[pat_id] = pat_dict.get(pat_id, 0)+1
#     print(pat_id, pat_dict[pat_id])
count = 0
count2 = 0
for items in (df['ISERROR']):
    if pd.isnull(items) == False:
        count2 += 1
    else:
        count += 1
print(count, count2)

99956 44


In [6]:
error_data = df[df.ISERROR == 1]
df = df[df.ISERROR.isna()]

We check the number of data entries that have chart events and the entries that don't.

In [7]:
patid_set = set()
patid = {}
for items in data:
    patid_set.add(items[2])
    patid[items[2]] = patid.get(items[2], 0)+1

In [8]:
# Code to check if multiple diagnosis occur for the same patient, if so how many times?
pat_diag_dict = {}
for items in data:
    pat_id = items[2]
    diag = items[26]
    if pat_id not in pat_diag_dict:
        pat_diag_dict[pat_id] = diag
    else:
        print(pat_diag_dict[pat_id], diag, items[3])
        
# Hence we conclude that there are cases when we have had different diagnosis for the same pati_id, hadm_id

In [9]:
# data = [{"pat_id":None}]
# data[0]['pat_id'] = 2
# data.append({'pat_id':3})
# l = 5
# desc = [items['pat_id'] for items in data]
# if l not in desc:
#     data.append({'pat_id':l})
# # print(data)
pat_set = set()
final_structure = []
for entry in data:
    pat_id = entry[2]
    stay_id = entry[3]
    diagnosis = entry[26]
    store_time = entry[6]
    if pat_id not in pat_set:
        # then we need to construct a new tuple with the entries
        pat_set.add(pat_id)
        new_struct = {'pat_id':pat_id, 'hospital':[]}

        stay_struct = {'stay_id':stay_id, 'diagnosis':[]} 
        diagnosis_struct = {'diagnosis':diagnosis, 'time':store_time}
        stay_struct['diagnosis'].append(diagnosis_struct)
        # we make diagnosis a list cause we might have differnt diagnosis for the same stay
        new_struct['hospital'].append(stay_struct)
        final_structure.append(new_struct)
    else:
        for items in final_structure:
            if pat_id == items['pat_id']:
                diagnosis_struct = {'diagnosis':diagnosis, 'time':store_time}
                items['hospital'][0]['diagnosis'].append(diagnosis_struct)
        # in this case it means there exists multiple cases of diagnosis for the same pat_id, hamd_id.
        
final_arr = [data['pat_id'] for data in final_structure]
print(final_structure[0])

{'pat_id': 1285489, 'hospital': [{'diagnosis': [{'time': '2182-06-10 17:28:00', 'diagnosis': '2182-06-09 15:45:00'}], 'stay_id': 1835}]}


In [10]:
import json

r = [{'is_claimed': 'True', 'rating': [100, 29, 30]},{'is_claimed': 'False', 'rating': [10, 2, 30]} ]
r = json.dumps(r)
loaded_r = json.loads(r)
print(loaded_r[0]['rating']) #Output 3.5


[100, 29, 30]
